<a href="https://colab.research.google.com/github/elonezra/deeplearningproject/blob/main/final_project_deep_learning_course.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Authors: Avi and Elon 🤗

In [49]:
# Importng necassary libraries
import pandas as pd
import numpy as np
import tensorflow.compat.v1 as tf
tf.compat.v1.disable_eager_execution()
import seaborn as sns
import matplotlib
import matplotlib.pyplot as plt
%matplotlib inline
import warnings

In [50]:
df = pd.read_csv('BTC-USD.csv')
df.head()

,Date,Open,High,Low,Close,Adj Close,Volume
0,2014-09-17,465.864014,468.174011,452.421997,457.334015,457.334015,21056800
1,2014-09-18,456.859985,456.859985,413.104004,424.440002,424.440002,34483200
2,2014-09-19,424.102997,427.834991,384.532013,394.795990,394.795990,37919700
3,2014-09-20,394.673004,423.295990,389.882996,408.903992,408.903992,36863600
4,2014-09-21,408.084991,412.425995,393.181000,398.821014,398.821014,26580100


In [51]:
# We don't need the "Adj Close" column because this column is the same as the "Close" column,
# we are going to check it before earasing to make sure our assumption is true.
print(df['Adj Close'].count(), df['Close'].count()) # In this line we can see that the number of lines in both of the columns is equal
print(len(df["Close"] == df["Adj Close"])) # and in this line we compared between those columns and get as result that all lines are equal
# thus, we can remove "Asj Close" column
df = df.drop(['Adj Close'], axis = 1)
df.head()

2713 2713
2713


,Date,Open,High,Low,Close,Volume
0,2014-09-17,465.864014,468.174011,452.421997,457.334015,21056800
1,2014-09-18,456.859985,456.859985,413.104004,424.440002,34483200
2,2014-09-19,424.102997,427.834991,384.532013,394.795990,37919700
3,2014-09-20,394.673004,423.295990,389.882996,408.903992,36863600
4,2014-09-21,408.084991,412.425995,393.181000,398.821014,26580100


In [52]:
# now we want to change the 'Close' column to be 'Open_Close_Mean' column because,
# it's more common to predict the mean between the Open price and the Close price
df['Close'].rename('Open_Close_Mean')
df['Open_Close_Mean'] = (df['Open'] + df['Close']) / 2
df = df.drop(['Close'], axis=1)

In [53]:
# In the code below we will see that our dataframe uses an object column which is not good
# because we need a numeric dataframe so, we convert this column to int64 type
# by removing the '-' char
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2713 entries, 0 to 2712
Data columns (total 6 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   Date             2713 non-null   object 
 1   Open             2713 non-null   float64
 2   High             2713 non-null   float64
 3   Low              2713 non-null   float64
 4   Volume           2713 non-null   int64  
 5   Open_Close_Mean  2713 non-null   float64
dtypes: float64(4), int64(1), object(1)
memory usage: 127.3+ KB


In [54]:
int(df.iloc[1][0].replace('-', ''))

C:\Users\rahim\AppData\Local\Temp\ipykernel_13388\84532947.py:1: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  int(df.iloc[1][0].replace('-', ''))


20140918

In [55]:
df["Date"] = df["Date"].apply(lambda x: x.replace('-','')).apply(pd.to_numeric)

In [56]:
# Now, after we convert the "Date" column to numeric type, we can see the new data as request
df.info()
df.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2713 entries, 0 to 2712
Data columns (total 6 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   Date             2713 non-null   int64  
 1   Open             2713 non-null   float64
 2   High             2713 non-null   float64
 3   Low              2713 non-null   float64
 4   Volume           2713 non-null   int64  
 5   Open_Close_Mean  2713 non-null   float64
dtypes: float64(4), int64(2)
memory usage: 127.3 KB


,Date,Open,High,Low,Volume,Open_Close_Mean
0,20140917,465.864014,468.174011,452.421997,21056800,461.599015
1,20140918,456.859985,456.859985,413.104004,34483200,440.649993
2,20140919,424.102997,427.834991,384.532013,37919700,409.449494
3,20140920,394.673004,423.295990,389.882996,36863600,401.788498
4,20140921,408.084991,412.425995,393.181000,26580100,403.453003


$\sigma = \sqrt(\mu_2) = \sqrt(1/n \sum(x_i - x')^2)$

$𝜇^{𝑙,𝑗}= \frac{1}{m} ∑_{𝑖=0}^{𝑚}{𝑧_𝑖^{𝑙,𝑗}}$ 

$𝜎^{𝑙,𝑗}=\sqrt{∑_{𝑖=0}^𝑚(𝑧_𝑖^{𝑙,𝑗}−𝜇^(𝑙,𝑗) )^2 }$

$(𝑧_𝑖^(𝑙,𝑗) ) ̂= \frac{𝑧_𝑖^{𝑙,𝑗}−𝜇^{𝑙,𝑗}}{𝜎^{𝑙,𝑗}+𝜖} $


In [57]:
def mu(data):
    return np.mean(data)/len(data)

def standard_deviation(data, mu):
  # Calculate the variance of the data.
  variance = np.sum((data - mu)**2)
  # Calculate the standard deviation of the data.
  standard_deviation = np.sqrt(variance)
  return standard_deviation


In [58]:
# As we can see, the data is not normalized so the next steps will focus on normalizing
# the data to numbers from 0 to 1 i.e. [0, 1]
epsilon = 0.00001

df = df.apply(lambda x: (x - mu(x)) / standard_deviation(x, mu(x))+epsilon)

# df = df.apply(lambda x: x/x.max())
df.head()

,Date,Open,High,Low,Volume,Open_Close_Mean
0,0.019172,0.000460,0.000451,0.000461,0.000022,0.000456
1,0.019172,0.000452,0.000440,0.000422,0.000032,0.000436
2,0.019172,0.000420,0.000412,0.000393,0.000035,0.000405
3,0.019172,0.000391,0.000408,0.000398,0.000034,0.000398
4,0.019172,0.000404,0.000398,0.000402,0.000026,0.000400


In [59]:
from sklearn.model_selection import train_test_split

# Assuming df is your DataFrame
# Replace 'target' with your target column name
X = df.drop(['Open_Close_Mean'], axis=1)
y = df['Open_Close_Mean']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

print("Training set size:", len(X_train))
print("Testing set size:", len(X_test))


Training set size: 2170
Testing set size: 543


In [60]:
# to extract the numebr of features in out dataset we need to index to the
# second element in the shape of the dataframe which is the number of columns
# in our dataframe and -1 for the features without the label
features = df.shape[1] - 1
x = tf.placeholder(tf.float32, [None, features])
y = tf.placeholder(tf.float32, [None, 1])
W = tf.Variable(tf.zeros([features,1]))
b = tf.Variable(tf.zeros([1]))
pred = tf.matmul(x,W) + b
loss = tf.reduce_mean(tf.pow(pred - y, 2))
update = tf.train.GradientDescentOptimizer(0.001).minimize(loss)
data_x = X_train
data_y = y_train.values.reshape(-1, 1)
sess = tf.Session()
sess.run(tf.global_variables_initializer())
for i in range(0,100000):
    sess.run(update, feed_dict = {x:data_x, y:data_y})
    if i % 10000 == 0 :
        print('Iteration:' , i , ' W:' , sess.run(W) , ' b:' , sess.run(b), ' loss:', loss.eval(session=sess, feed_dict = {x:data_x, y:data_y}))


Iteration: 0  W: [[4.2202660e-07]
 [7.2788862e-07]
 [7.2782632e-07]
 [7.2721195e-07]
 [5.8926577e-07]]  b: [2.1945376e-05]  loss: 0.00036332908


Iteration: 10000  W: [[0.00021181]
 [0.00494658]
 [0.00494587]
 [0.00493932]
 [0.00354226]]  b: [0.01077623]  loss: 0.0002350416
Iteration: 20000  W: [[0.00021283]
 [0.00969042]
 [0.00968903]
 [0.00967635]
 [0.00689832]]  b: [0.01058275]  loss: 0.00022716718
Iteration: 30000  W: [[0.0002138 ]
 [0.01435432]
 [0.01435213]
 [0.01433334]
 [0.01019153]]  b: [0.01039274]  loss: 0.00021956058
Iteration: 40000  W: [[0.00021467]
 [0.01893914]
 [0.01893614]
 [0.01891185]
 [0.01342291]]  b: [0.01020579]  loss: 0.00021221294
Iteration: 50000  W: [[0.00021566]
 [0.02344667]
 [0.02344283]
 [0.02341327]
 [0.01659356]]  b: [0.01002202]  loss: 0.0002051151
Iteration: 60000  W: [[0.00021653]
 [0.02787806]
 [0.02787334]
 [0.0278387 ]
 [0.01970472]]  b: [0.00984167]  loss: 0.00019825868
Iteration: 70000  W: [[0.0002174 ]
 [0.03223315]
 [0.03222775]
 [0.03218937]
 [0.02275685]]  b: [0.00966418]  loss: 0.00019163694
Iteration: 80000  W: [[0.00021827]
 [0.03651616]
 [0.03650976]
 [0.03646666]
 [0.02575159]] 

In [61]:
from sklearn.metrics import mean_squared_error
from sklearn.metrics import r2_score

b = tf.cast(b, tf.float32)  # Ensure b is float32
X_test_np = X_test.to_numpy().astype('float32')  # Convert X_test to numpy array of type float32

# Compute y_pred
y_pred_tensor = tf.matmul(X_test_np, W) + b  # Now X_test, W, and b are of the same type
y_pred = sess.run(y_pred_tensor)

# Calculate RMSE
rmse = np.sqrt(mean_squared_error(y_test, y_pred))

# Calculate the range of the target variable
range_y = np.max(y_test) - np.min(y_test)

# Calculate NRMSE
nrmse = rmse / range_y

# Assuming y_test are the true target values and y_pred are the predicted target values
r2 = r2_score(y_test, y_pred)

print("R-squared: ", r2)

r2_percentage = r2 * 100

print("R-squared (%): ", r2_percentage)

print("NRMSE: ", nrmse)


R-squared:  0.28971806204698125
R-squared (%):  28.971806204698126
NRMSE:  0.21182192455149204


In [62]:
%load_ext tensorboard
# Define the number of features
features = df.shape[1] - 1

# Define placeholders
x = tf.placeholder(tf.float32, [None, features])
y = tf.placeholder(tf.float32, [None, 1])

# Define weights and bias
W1 = tf.Variable(tf.random_normal([features, 10]))
b1 = tf.Variable(tf.zeros([10]))
W2 = tf.Variable(tf.random_normal([10, 1]))
b2 = tf.Variable(tf.zeros([1]))

# Define the model
hidden_layer = tf.nn.relu(tf.matmul(x, W1) + b1)
pred = tf.matmul(hidden_layer, W2) + b2

# Define the loss function (Mean Squared Error)
loss = tf.reduce_mean(tf.square(pred - y))

# Define the optimizer (Gradient Descent)
optimizer = tf.train.GradientDescentOptimizer(0.001).minimize(loss)

# Initialize variables
init = tf.global_variables_initializer()

# Reshape y_train and y_test to be 2D arrays
y_train = y_train.values.reshape(-1, 1)
y_test = y_test.values.reshape(-1, 1)

# Start a Session
with tf.Session() as sess:
    sess.run(init)

    # Train the model
    for epoch in range(10000):
        _, c = sess.run([optimizer, loss], feed_dict={x: X_train, y: y_train})

        # Print loss every 1000 epochs
        if epoch % 1000 == 0:
            print("Epoch:", '%04d' % (epoch), "cost=", "{:.9f}".format(c))

    # Make predictions
    y_pred = sess.run(pred, feed_dict={x: X_test})

# Calculate R-Squared score
r2 = r2_score(y_test, y_pred)
print("R-Squared: ", r2)

r2_percentage = r2 * 100

print("R-squared (%): ", r2_percentage)

print("NRMSE: ", nrmse)


The tensorboard extension is already loaded. To reload it, use:
  %reload_ext tensorboard
Epoch: 0000 cost= 0.000294410
Epoch: 1000 cost= 0.000147841
Epoch: 2000 cost= 0.000141442
Epoch: 3000 cost= 0.000137725
Epoch: 4000 cost= 0.000135190
Epoch: 5000 cost= 0.000133333
Epoch: 6000 cost= 0.000131931
Epoch: 7000 cost= 0.000130677
Epoch: 8000 cost= 0.000129490
Epoch: 9000 cost= 0.000128366
R-Squared:  -2.0417985586076646
R-squared (%):  -204.17985586076645
NRMSE:  0.21182192455149204


this is an improve code to view the graph model